# Parameters and constructions

In [162]:
p = 2
m = 1
q = p^m
K.<alpha> = GF(q)
N = 15
L_N.<a> = GF(q^N)
P = L_N._modulus

In [163]:
x = ['x'+str(k) for k in range(N)]
M = PolynomialRing(L_N,N,x)
I = M.ideal([M(x[i])^q - M(x[i]) for i in range(N)])
Q = M.quotient(I)

R.<X> = PolynomialRing(M)
S = R.quotient(R(P))

variable22 = sum([Q(x[i])*a^i for i in range(N)])

def HFE_representation_polynome(f):
    """
    Param: f, a polynomial in L_N[X] 
    Return: (p0,p1,..,pn), polynomials in Fq[x0,..,xN-1] such that 
            f(x0,..,xn) = (p0(x0,..,xn),.., pn(x0,..xn))
    """
    p_list = vector([0]*N)
    for i in f(variable22).lift():
        p_list += vector(i[0])*i[1]
 
    return p_list

# HFE Cryptosystem

In [164]:
def HFE_function(d, number_coeff):
    """
    Param: d, degree of f 
            number_coeff, number of coefficient nonzero of f
    Sortie: f, polynomial which verify the HFE conditions
    """
    
    coeff = [L_N.random_element() for i in range(number_coeff)]
    f = 0
    lim = abs(floor(log(d,q)-1))
    for i in range(len(coeff)-2):
        theta = randint(0,lim)
        phi = randint(0,lim)
        f += coeff[i] * X^(q^theta + q^phi)
        
    mu_0 = L_N.random_element()
    f += mu_0 + coeff[-1] * X^d 
    return f

In [165]:
def HFE_generates_keys(d, number_coeff):
    f = HFE_function(d, number_coeff)
    
    T.<Y> = PolynomialRing(L_N)

    s = R(T.random_element(1))
    t = R(T.random_element(1))
    
    p_list = HFE_representation_polynome(s(f(t)))
    private_key = (L_N,f,s,t)
    public_key = (K, p_list)
    return public_key, private_key 

In [166]:
def HFE_encryption(m, public_key):
    """
    Param: m, an element of (F_q)^N
    Return: c, c = HFE(m)
    """
    K, p_list = public_key
    m = tuple(m)
    return [p(m) for p in p_list]

In [167]:
def HFE_decryption(y, private_key):
    L_N, f, s, t = private_key
    a = L_N([0,1])
    y = sum([y[i]*a**i for i in range(N)])

    list_s = list(s)
    s_inv = list_s[1]**(-1) * (X - list_s[0])
    list_t = list(t)
    t_inv = list_t[1]**(-1) * (X - list_t[0])
        
    ft = s_inv(y)
    
    v = tuple((L_N^N).zero())
    
    roots = (f-ft).roots(multiplicities = False)
    m = [L_N.vector_space(map = false)((t_inv(r))(v)) for r in roots]
   
    return m

## Example and verification

In [168]:
t = cputime()
public_key, private_key = HFE_generates_keys(10,9)
cputime(t)

0.08805200000000468

In [169]:
private_key

(Finite Field in a of size 2^15,
 ((a^14 + a^11 + a^7 + a^4 + a^2 + a))*X^10 + ((a^14 + a^13 + a^11 + a^7 + a^6 + a^5 + a^4 + a^3 + a))*X^8 + ((a^14 + a^13 + a^11 + a^10 + a^9 + a^7 + a^6 + a^5 + a^2 + 1))*X^5 + ((a^12 + a^11 + a^9 + a^7 + a^6 + a^5 + a^3 + a^2 + a))*X^4 + ((a^12 + a^11 + a^9 + a^8 + a^6 + a^5 + a^3 + a^2 + a))*X^3 + ((a^14 + a^13 + a^9 + a^8 + a^6 + a^5 + a^4 + a^3 + a))*X^2 + (a^14 + a^13 + a^7 + a^6 + a^5 + a^4 + a^3 + a^2 + a),
 ((a^11 + a^9 + a^7 + a^4 + a))*X + (a^14 + a^13 + a^11 + a^10 + a^9 + a^6 + a^4 + a^3),
 ((a^14 + a^13 + a^12 + a^10 + a^7 + a^5 + a^4 + 1))*X + (a^14 + a^13 + a^11 + a^10 + a^7 + a^5 + a^3 + a^2 + a))

In [186]:
plaintext = L_N.vector_space(map=False)(L_N.random_element())
cyphertext = HFE_encryption(plaintext,public_key)
m = HFE_decryption(cyphertext, private_key)
m

[(0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0)]

In [187]:
plaintext 

(0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0)

# Cryptanalysis with Gröbner basis

In [188]:
def solve_sys(S,B):
    def solve_sys_rec(S,expens,i):
        expension = expens.copy()
        if i == 0:
            return expension
        Scopy = S.copy()
        f = Scopy[i-1]
        if f == 0:
            return solve_sys_rec(Scopy,expension,i-1)
        f = f.univariate_polynomial()
        root = f.roots(multiplicities = False)
        key = f.variable_name()
        if (len(root) == 1):
            r = root[0]
            new_S = []
            expension[B(key)] = r
            for s in Scopy:
                new_S += [s.subs(expension)] 
            return solve_sys_rec(new_S,expension,i-1)
        res = []
        for r in root:
            new_S = []
            expension[B(key)] = r
            for s in Scopy:
                new_S += [s.subs(expension)] 
            res += [solve_sys_rec(new_S,expension,i-1)]        
        return res
    expension = {B(xi):B(xi) for xi in x}
    l = solve_sys_rec(S, expension, len(S))
    return (l)

In [189]:
def HFE_cryptanalysisI(public_key,p):
    K,Sys = public_key
    N = len(Sys)
    x = Sys[0].variables()
    B = PolynomialRing(K,N,x, order = 'lex')
    Sys = [Sys[i] - p[i] for i in range(N)] + [B(xi)^2-B(xi) for xi in x]
    IS = Ideal(B,Sys)
    Sols = IS.groebner_basis()
    return solve_sys(Sols,B)

In [174]:
def HFE_cryptanalysisII(public_key,p):
    K,Sys = public_key
    N = len(Sys)
    x = Sys[0].variables()
    B = PolynomialRing(K,N,x)
    Sys = [Sys[i] - p[i] for i in range(N)] + [B(xi)^2-B(xi) for xi in x]
    IS = Ideal(B,Sys)
    Sols = IS.groebner_basis()
    ISols = Sols.ideal()
    P = PolynomialRing(K,N,x,order = 'lex')
    I = list(ISols.transformed_basis('fglm',P))
    return solve_sys(I,P)

Give a recursive list of dict, we could give a better representation with vector

In [190]:
t = cputime()
mI = (HFE_cryptanalysisI(public_key,cyphertext))
print(cputime(t))

12.069910000000021


In [191]:
t = cputime()
mII = (HFE_cryptanalysisII(public_key,cyphertext))
print(cputime(t))

0.39731199999999944


In [192]:
mI == mII

True

In [193]:
mI

{x0: 0,
 x1: 0,
 x2: 0,
 x3: 0,
 x4: 0,
 x5: 0,
 x6: 1,
 x7: 0,
 x8: 1,
 x9: 1,
 x10: 1,
 x11: 0,
 x12: 1,
 x13: 0,
 x14: 0}

In [194]:
plaintext

(0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0)